In [1]:
import import_ipynb
import numpy as np
import sys
import csv
import numpy.ma as ma
import matplotlib.pyplot as plt
import a2_extract_list_meetings as a2
import c1_extract_paragraphe_issue as c1
import c2_extract_sentence_issue as c2
import e1_extract_occ_gen as e1
import e2_extract_occ_original as e2 
import g1_generate_dictionary as g1



importing Jupyter notebook from a2_extract_list_meetings.ipynb
importing Jupyter notebook from a1_compute_list_meeting.ipynb
importing Jupyter notebook from c1_extract_paragraphe_issue.ipynb
importing Jupyter notebook from c2_extract_sentence_issue.ipynb
importing Jupyter notebook from g1_generate_dictionary.ipynb
importing Jupyter notebook from e2_extract_occ_original.ipynb
importing Jupyter notebook from e1_extract_occ_gen.ipynb


In [2]:
def open_file(file):
    list_file = []
    f = open(file)
    for x in csv.reader(f):
        list_file.append(x)
    return list_file[1:]

In [6]:
def write_occurrence_issue(occurences_meetings, s):
        with open(s, "w", newline='') as file:
                writer = csv.writer(file)
        #header
                writer.writerow(('entity','interventions'))
                writer.writerows(occurences_meetings)

In [5]:
def combine_gen(dictionary, original_occ, generated_occ):
    combined = []

    for eo in original_occ:
        list_eo = []
        for eg in generated_occ:
            if(eg[0] in dictionary[eo[0]]):
                list_eo.append(eg)
        combined.append(list_eo)
    return combined

In [7]:
def create_stat_egals(generated_occ, original_occ,number):
    original = []
    generated = []
    for i in range(len(original_occ)):
        original.append((number,original_occ[i][0],original_occ[i][1]))
        g = sorted(generated_occ[i],key=lambda x: x[1], reverse=True)[0]
        generated.append((number,g[0],g[1]))
    return original,generated 

In [8]:
def extract_interventions(number, list_sentences):
    """Extract the intervention for both dataset and keep only the one that match the dictionary"""
    #Load the occurences for the original and generated file, and the dictionary
    original_occ = e2.extract_occurences_issue(number)
    gen_occ = e1.extract_occurences_issue_ENB(list_sentences)

    dictionary = g1.compute_dictionary()

    #Combine element same key
    generated_occ= combine_gen(dictionary, original_occ, generated_occ)
    # Original and generated occurence with only the entities into the dictionary
    original,generated = create_stat_egals(generated_occ, original_occ, number)

    return original,generated 

In [10]:
def extract__all_intervention():
    """Extract the intervention for all the issues in the original dataset and write them into .csv file"""
    # Extract all the issue number in the original dataset
    list_meetings = open_file('list_meetings.csv')
    list_issues = a2.issues_list(list_meetings)

    list_generated = []
    list_original = []

    for i in list_issues:
        if(45 < int(i)):
            list_paragraphes = c1.extract_paragraphes_from_issue(int(i))
            list_sentences = c2.sentences(list_paragraphes)
            original,generated = extract_interventions(i, list_sentences)
            list_generated += generated
            list_original += original
    write_occurrence_issue(list_generated,"occurences_generated.csv")
    write_occurrence_issue(list_original,"occurences_original.csv")
    

In [11]:
find_intervention()

Issue  46
Issue  47
Issue  48
Issue  49
Issue  50
Issue  51
Issue  52
Issue  53
Issue  54
Issue  55
Issue  56
Issue  57
Issue  58
Issue  59
Issue  60
Issue  61
Issue  62
Issue  63
Issue  64
Issue  65
Issue  66


KeyboardInterrupt: 

import csv
def open_list_meetings():
    """ Open the csv file that contain all the meetings. """
    f = open('list_meetings.csv')
    return csv.reader(f)


def define_dict_ref():
    list_ref = list(dict.fromkeys([(i[0],i[1]) for i in open_list_meetings()]))[1:]
    dict_ref = dict()
     
    k =0
    for r in list_ref:
        s = str(k)
        k +=1
        dict_ref[s] = []
        for line in open_list_meetings():
           if((line[0],line[1]) == r):
               dict_ref[s].append(line[4])
       
    return dict_ref



def extract_occurence_by_list_ref(dict_ref):
    generated = open_file("occurences_generated.csv")
    occ_ch = []
    occ_us = []
    occ_china = []
    occ_sa = []
    num = 0
    print(len(dict_ref.keys()))
    for k in dict_ref.keys():
        o_ch = 0
        o_us = 0
        o_china = 0
        o_sa = 0
 
        for line in generated:
            if(line[0] in dict_ref[k] and (line[1] == 'SWITERLAND' or line[1] == 'SWITZER' or line[1] == 'SWITZERLAND')):
                o_ch += int(line[2])
                    
            if(line[0] in dict_ref[k] and (line[1] == 'US')):
                o_us += int(line[2])
            if(line[0] in dict_ref[k] and (line[1] == 'CHINA')):
                o_china += int(line[2])
            if(line[0] in dict_ref[k] and (line[1] == 'SAUDI ARABIA' or line[1]== 'SAUDIA ARABIA' )):
                o_sa += int(line[2])
    
        if(10 < int(k) and int(k) < 105):
                occ_ch.append(o_ch)
                occ_us.append(o_us)
                occ_china.append(o_china)
                occ_sa.append(o_sa)
                print('coucou')
    return occ_ch, occ_us, occ_china, occ_sa
        

d = define_dict_ref()

swiss, us, china, saudi_arabia= extract_occurence_by_list_ref(d)





import pandas as pd

index = np.arange(len(us))
df = pd.DataFrame({'Switzerland': swiss,'China': china, 'US': us, 'Saudi Arabia': saudi_arabia}, index=index)

ax = df.plot(figsize=(25,9),kind='bar',title='Number of intervention for each meetings between 1995-2013')
ax.set_xlabel("Meetings in chronological order")
ax.set_ylabel("Number of interventions")
